In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
opt = SolverFactory('glpk').executable = 'D:\download\glpk-4.65\w64'

In [2]:
#Read generators and demand data as two dataframes  Desktop717data
df_genData=pd.read_excel("Desktop/717/data/S1try1.xlsx",sheet_name="GeneratorsData",skiprows=1) #read data from generators tab and skip the first row
df_AverloadsData=pd.read_excel("Desktop/717/data/S1try1.xlsx",sheet_name="rooftopS1AverageDemand2040",skiprows=1)
#df_PeakloadsData=pd.read_excel("Desktop/717/data/S1try1.xlsx",sheet_name="PeakLoadData",skiprows=1) 
df_linesData=pd.read_excel("Desktop/717/data/S1try1.xlsx",sheet_name="LinesData",skiprows=1) 

In [3]:
#define the size of the sets
NumGens=len(df_genData)
NumNodes=len(df_AverloadsData)
NumLines=len(df_linesData)
#T=len(df_demandData)
print("We read data for", NumGens, "power generating units")
print("We read data for", NumNodes, "nodes")
print("We read data for", NumLines, "transmission lines")

We read data for 32 power generating units
We read data for 3 nodes
We read data for 3 transmission lines


In [4]:
#Define Indexes
G=np.array([g for g in range(0,NumGens)])  
N=np.array([n for n in range(0,NumNodes)])
L=np.array([l for l in range(0,NumLines)])

In [5]:
df_genData.columns

Index(['GenName', 'Node', 'FixedCost', 'SDCost', 'SUCost', 'VarCost', 'MaxGen',
       'PSpringMaxGen', 'PSummerMaxGen', 'PAutumnMaxGen', 'PWinterMaxGen',
       'OSpringMaxGen', 'OSummerMaxGen', 'OAutumnMaxGen', 'OWinterMaxGen',
       'MinGen', 'RampDown', 'RampSD', 'RampSU', 'RampUp', 'MinUpTime',
       'MinDownTime', 'ReqUp', 'ReqDown', 'InitialStatus', 'InitialGen',
       'Plant Name', 'Fuel Type', 'Average Emission Rate'],
      dtype='object')

In [6]:
##Declare the generator's parameters and assign them the data we read before
VarCost=df_genData.loc[:,'VarCost'].to_numpy()
# generation capacity of peak hour for each seasons
PSpringPMax=df_genData.loc[:,'PSpringMaxGen'].to_numpy()
PSummerPMax=df_genData.loc[:,'PSummerMaxGen'].to_numpy()
PFallPMax=df_genData.loc[:,'PAutumnMaxGen'].to_numpy()
PWinterPMax=df_genData.loc[:,'PWinterMaxGen'].to_numpy()
# generation capacity of off-peak hour for each seasons
OSpringPMax=df_genData.loc[:,'OSpringMaxGen'].to_numpy()
OSummerPMax=df_genData.loc[:,'OSummerMaxGen'].to_numpy()
OFallPMax=df_genData.loc[:,'OAutumnMaxGen'].to_numpy()
OWinterPMax=df_genData.loc[:,'OWinterMaxGen'].to_numpy()
#emission rate of each generator
GEmissionRate=df_genData.loc[:,'Average Emission Rate'].to_numpy()
#other attributes
PMin=df_genData.loc[:,'MinGen'].to_numpy()
GenNode=df_genData.loc[:,'Node'].to_numpy()

In [7]:
df_linesData.columns  #In the data we defined 3 lines as: L1-2, L2-3, and L3-1.
#We could have defined them in other way, but this is convenient for the KVL equation

Index(['LineName', 'NodeFrom', 'NodeTo', 'Reactance', 'Capacity'], dtype='object')

In [8]:
##Declare the transmission lines' parameters and assign them the data we read before
NodeFrom=df_linesData.loc[:,'NodeFrom'].to_numpy()
NodeTo=df_linesData.loc[:,'NodeTo'].to_numpy()
LineReactance=df_linesData.loc[:,'Reactance'].to_numpy()
LineCapacity=df_linesData.loc[:,'Capacity'].to_numpy()

In [9]:
df_AverloadsData.columns

Index(['LoadName', 'Node', 'PeakWinter', 'PeakSpring', 'PeakSummer',
       'PeakFall', 'Off-PeakWinter', 'Off-PeakSpring', 'Off-PeakSummer',
       'Off-PeakFall'],
      dtype='object')

In [10]:
ALoadBus=df_AverloadsData.loc[:,'Node'].to_numpy( )
#Demand of peak hours and off-peak hour in each seasons
APeakSpringDemand=df_AverloadsData.loc[:,'PeakSpring'].to_numpy( )
AOffPeakSpringDemand=df_AverloadsData.loc[:,'Off-PeakSpring'].to_numpy( )
APeakSummerDemand=df_AverloadsData.loc[:,'PeakSummer'].to_numpy( )
AOffPeakSummerDemand=df_AverloadsData.loc[:,'Off-PeakSummer'].to_numpy( )
APeakFallDemand=df_AverloadsData.loc[:,'PeakFall'].to_numpy( )
AOffPeakFallDemand=df_AverloadsData.loc[:,'Off-PeakFall'].to_numpy( )
APeakWinterDemand=df_AverloadsData.loc[:,'PeakWinter'].to_numpy( )
AOffPeakWinterDemand=df_AverloadsData.loc[:,'Off-PeakWinter'].to_numpy( )


In [11]:
# define a function indicatorMatrix that creates indicator matrices to relate generators with their node
# and relate lines with the nodes
#dataRows is a column vector of NumRows rows. Each element says the column where this row should be a one in the final matrix
def IndicatorMatrix(NumRows, NumCols, dataRows):
    matrix= np.zeros((NumRows, NumCols),dtype=int)
    for i in range(0,NumRows):
        matrix[i,dataRows[i]-1]=1
    return matrix

In [12]:
#Create indicator matrices using the function defined above
#GeneratorInBus is a matrix of G rows and N columns that has 1 in the position g,n if generator g is AT node N, 0 otherwise
GeneratorInBus=IndicatorMatrix(NumGens,NumNodes,GenNode)
#LineFromBus is an indicator matrix of L rows and N columns that has 1 in the position l,n if line l departs FROM node N, 0 otherwise
LineFromBus=IndicatorMatrix(NumLines,NumNodes,NodeFrom)
#LineToBus is an indicator matrix of L rows and N columns that has 1 in the position l,n if line l arrives TO node N, 0 otherwise
LineToBus=IndicatorMatrix(NumLines,NumNodes,NodeTo)

In [13]:
GeneratorInBus

array([[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]])

In [99]:
#UC MODEL of Average winter peak hour
def APeakWinterED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,10000))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PWinterPMax[g])
    #m.MaxEmission= Constraint(m.G, rule=lambda m, g:  sum(m.p[g]* (GEmissionRate[g])for g in m.G) <= 4000000)#maybe put parameter to replace exact number
    # refer Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)
   
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            == APeakWinterDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m





In [100]:
m=APeakWinterED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 245179.984127341
  Upper bound: 245179.984127341
  Number of objectives: 1
  Number of constraints: 43
  Number of variables: 36
  Number of nonzeros: 80
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04955458641052246
# ----------------------------------------------------------
#   Solution Information
# ------------

In [102]:
# create an empty DataFrame with the appropriate column names
dfeachgen = pd.DataFrame(columns=["Generator", "Power Output"])

for g in m.G:
    dfeachgen.loc[g, "Generator"] = g
    dfeachgen.loc[g, "Power Output"] = m.p[g].value


TotalEmission= dfeachgen['Power Output']*df_genData['Average Emission Rate']

sum(TotalEmission)

7446937.611736137

In [101]:
print('SOLUTION-Average winter peak')
print('The total system cost is = $',m.system_cost())
#print("The total emission in this hour ={1:.2f}".format(sum(TotalEmission)*0.000453592))
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average winter peak
The total system cost is = $ 245179.9841273406
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 0.00 
generator  4 = 0.00 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 0.00 
generator  8 = 0.00 
generator  9 = 0.00 
generator 10 = 0.00 
generator 11 = 0.00 
generator 12 = 0.00 
generator 13 = 0.00 
generator 14 = 0.00 
generator 15 = 809.72 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 0.00 
generator 19 = 0.00 
generator 20 = 503.57 
generator 21 = 0.00 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 6060.61 
generator 28 = 490.84 
generator 29 = 490.84 
generator 30 = 368.00 
generator 31 = 0.00 
generator 32 = 0.00 
Flow on transmission lines in MW
line  1 = 900.00 
line  2 = 0.00 
line  3 = -900.00 
LMPs in $/MWh
Node  1 = 0.00
Node  2 = 30.00
Node  3 = 25.00
